# | default_exp core

In [ ]:
# | hide
from bertopic import BERTopic
from bertopic.vectorizers import OnlineCountVectorizer
import dagshub
from datetime import datetime
import dill as pickle
import dvc.api
from hdbscan import HDBSCAN
from itertools import tee, islice
import mlflow
import nbdev
from nbdev.showdoc import *
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import (
    CountVectorizer
    , TfidfTransformer
    , TfidfVectorizer
    ,
)
from sklearn.pipeline import make_pipeline
from src.custom_stanza_mlflow import StanzaWrapper
import src.dataframe_preprocessor as dfpp
import stanza
from tqdm import tqdm
from umap import UMAP

In [ ]:
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128'

# | export
def foo():
    pass

In [ ]:
# | hide
# this function allows us to get the experiment ID from an experiment name
def get_experiment_id(name):
    exp = mlflow.get_experiment_by_name(name)
    if exp is None:
      exp_id = mlflow.create_experiment(name)
      return exp_id
    return exp.experiment_id

In [ ]:
def custom_analyzer(step_list, stanza_pipeline, minNgramLength, maxNgramLength, lemmatize=True):
    lowered = " brk ".join(map(str, [step for step in step_list if step is not None])).lower()

    preproc = stanza_pipeline(lowered)
    
    if lemmatize:
        lemmad = " ".join(map(str,
                            [word.lemma
                            for sent in preproc.sentences 
                            for word in sent.words if (
                                word.upos not in ["NUM", "DET", "ADV", "CCONJ", "ADP", "SCONJ", "PUNCT"]
                                and word is not None
                            )]
                        )
                    )
    else:
        lemmad = " ".join(map(str,
                            [word.text
                            for sent in preproc.sentences 
                            for word in sent.words if (
                                word is not None
                            )]
                        )
                    )
    # analyze each line of the input string seperately
    for ln in lemmad.split(' brk '):
        # tokenize the input string (customize the regex as desired)
        at_least_two_english_characters_whole_words = "(?u)\b[a-zA-Z]{2,}\b"
        terms = re.split(at_least_two_english_characters_whole_words, ln)

        # loop ngram creation for every number between min and max ngram length
        for ngramLength in range(minNgramLength, maxNgramLength+1):

            # find and return all ngrams
            # for ngram in zip(*[terms[i:] for i in range(3)]): 
                # <-- solution without a generator (works the same but has higher memory usage)
            for ngram in zip(*[islice(seq, i, len(terms)) for i, seq in enumerate(tee(terms, ngramLength))]):   # <-- solution using a generator
                
                ngram = ' '.join(map(str, ngram))
                # yield ngram
                return str(ngram)


In [ ]:
# | hide
nbdev.nbdev_export()

# | Below this are blocks to use DagsHub with MLflow

In [ ]:
#@markdown Enter the username of your DAGsHub account:
DAGSHUB_USER_NAME = "AaronWChen" #@param {type:"string"}

#@markdown Enter the email for your DAGsHub account:
DAGSHUB_EMAIL = "awc33@cornell.edu" #@param {type:"string"}

#@markdown Enter the repo name 
DAGSHUB_REPO_NAME="MeaLeon"

#@markdown Enter the name of the branch you are working on 
BRANCH="STANZA-2/investigate_bertopic_compatibility"
dagshub.init(repo_name=DAGSHUB_REPO_NAME
             , repo_owner=DAGSHUB_USER_NAME)


Repository initialized!

In [ ]:
# instantiate stanza pipeline
stanza.download('en')
nlp = stanza.Pipeline('en', 
                      depparse_batch_size=50, 
                      depparse_min_length_to_batch_separately=50,
                      verbose=True,
                      use_gpu=False,
                    #   batch_size=100
                      )

2023-12-18 19:10:46 INFO: Downloading default packages for language: en (English) ...
2023-12-18 19:10:47 INFO: File exists: /home/awchen/stanza_resources/en/default.zip
2023-12-18 19:10:50 INFO: Finished downloading models and saved to /home/awchen/stanza_resources.
2023-12-18 19:10:50 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-12-18 19:10:51 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |
| sentiment    | sstplus             |
| ner          | ontonotes_charlm    |

2023-12-18 19:10:51 INFO: Using device: cpu
2023-12-18 19:10:51 INFO: Loading: tokenize
2023-12-18 19:10:51 INFO: Loading: pos
2023-12-18 19:10:51 INFO: Loading: lemma
2023-12-18 19:10:51 INFO: Loading: constituency
2023-12-18 19:10:52 INFO: Loading: depparse
2023-12-18 19:10:52 INFO: Loading: sentiment
2023-12-18 19:10:52 INFO: Loading: ner
2023-12-18 19:10:52 INFO: Done loading processors!


In [ ]:
mlflow.set_tracking_uri(f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# starter idea for making an experiment name can be the git branch, but need more specificity
experiment_name = f"{DAGSHUB_EMAIL}/bertopic_stanza_small_set_v1"
mlflow_exp_id = get_experiment_id(experiment_name)

# run_id that has the logged info needed
mlflow_run_id = 'c6fbcf396af34ee3aade5503ee01c2bb'

In [ ]:
# load from MLflow
mlflow_client = mlflow.tracking.MlflowClient(
    tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# load dataframes from artifacts
# mlflow.artifacts.download_artifacts(
#     run_id=mlflow_run_id
# )

# cv_params are parameters for the sklearn CountVectorizer or TFIDFVectorizer
cv_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
    'min_df':3,
}

# bertopic_params are a superset of cv_params
bertopic_params = {
    'top_n_words':20,
    'min_topic_size':5,
    'nr_topics':'auto',
    'verbose':True,
    'low_memory':True,
    'calculate_probabilities':True
}

# update bertopic_params to include cv_params
# bertopic_params.update(cv_params)

# pipeline_params are parameters that will be logged in MLFlow and are a superset of library parameters
pipeline_params = {
    'stanza_model': 'en',
    'sklearn-transformer': 'TfidfVectorizer'
}

# update the pipeline parameters with the library-specific ones so that they show up in MLflow Tracking
pipeline_params.update(cv_params)
pipeline_params.update(bertopic_params)

with mlflow.start_run(experiment_id=mlflow_exp_id):    
    # LOG PARAMETERS
    mlflow.log_params(pipeline_params)

    # LOG INPUTS (QUERIES) AND OUTPUTS
    # MLflow example uses a list of strings or a list of str->str dicts
    
    # load raw data and preprocess/clean
    data = dvc.api.read(
           path='../data/recipes-en-201706/epicurious-recipes_m2.json'
           , mode='r')
    raw_df = pd.read_json(data)
    print('\n')
    print('--------------')
    print('Raw Dataframe:', end='\n')
    print(raw_df.head())
    print(raw_df.shape)

    # pre_proc_df is cleaned dataframe
    pre_proc_df = dfpp.preprocess_dataframe(raw_df)
    print('\n')
    print('--------------')
    print('Preprocessed Dataframe:', end='\n')
    print(pre_proc_df.head())
    print(pre_proc_df.shape)


    # pre_proc_df = pd.read_json(
    #     mlflow.artifacts.download_artifacts(
    #         run_id=mlflow_run_id,
    #         artifact_path='artifacts/preprocessed_dataframes/preprocessed_dataframe.json',
    #         # tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow'
    #     )
    # )
    # print('\n')
    # print('-' * 80)
    # print('Preprocessed Dataframe:', end='\n')
    # print(pre_proc_df.head())
    # print(pre_proc_df.shape)

    # create subset for dev purposes
    to_nlp_df = pre_proc_df[0:50]
    print('\n')
    print('-' * 80)
    print('Subset Dataframe:', end='\n')
    print(to_nlp_df.head())
    print(to_nlp_df.shape)

    # LOG MODEL
    # Instantiate BERTopic
    topic_model = BERTopic(
        **bertopic_params,
    )

    def custom_analyzer(step_list, stanza_pipeline, minNgramLength, maxNgramLength):
            lowered = " brk ".join(map(str, [step for step in step_list if step is not None])).lower()

            preproc = stanza_pipeline(lowered)
            
            lemmad = " ".join(map(str,
                                [word.text
                                for sent in preproc.sentences 
                                for word in sent.words if (
                                    word is not None
                                )]
                            )
                        )
            
            # analyze each line of the input string seperately
            for ln in lemmad.split(' brk '):
                
                # tokenize the input string (customize the regex as desired)
                at_least_two_english_characters_whole_words = "(?u)\b[a-zA-Z]{2,}\b"
                terms = re.split(at_least_two_english_characters_whole_words, ln)

                # loop ngram creation for every number between min and max ngram length
                for ngramLength in range(minNgramLength, maxNgramLength+1):

                    # find and return all ngrams
                    # for ngram in zip(*[terms[i:] for i in range(3)]): 
                        # <-- solution without a generator (works the same but has higher memory usage)
                    for ngram in zip(*[islice(seq, i, len(terms)) for i, seq in enumerate(tee(terms, ngramLength))]):   # <-- solution using a generator
                        
                        ngram = ' '.join(map(str, ngram))
                        # yield ngram
                        return str(ngram)

    analyzer_kwargs = {'stanza_pipeline': nlp
                       , 'minNgramLength': 1
                       , 'maxNgramLength': 4}
    
    recipe_steps = to_nlp_df["prepSteps"].apply(custom_analyzer, **analyzer_kwargs)

    # recipe_steps = "".join(str(to_nlp_df["prepSteps"].apply(StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4))))
    print('\n')
    print('-' * 80)
    print('Recipe steps:', end='\n')
    print(recipe_steps)

    # train on the recipes' steps
    topics, probs = topic_model.fit_transform(recipe_steps)

    # since this uses a custom Stanza analyzer, we have to use a custom mlflow.Pyfunc.PythonModel
    # Instantiate sklearn CountVectorizer
    # steps_vectorizer_model = CountVectorizer(**cv_params)

    # May need to use BERTopic's OnlineCountVectorizer
    steps_vectorizer_model = OnlineCountVectorizer(**cv_params)

    # Do fit transform on data
    # steps_test_tfidf_transform = steps_tfidf_vectorizer_model.fit_transform(tqdm(to_nlp_df["steps"]))
    topic_model.update_topics(
        recipe_steps
        , vectorizer_model=steps_vectorizer_model
    )

    # Display topic model results
    print('\n')
    print('-' * 80)
    print('BERTopic Model Dataframe:', end='\n')
    print(topic_model.get_topic_info())

    print('\n')
    print('-' * 80)
    print('BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representation'])

    print('\n')
    print('-' * 80)
    print('BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representative_Docs'])

    # Save and log the topic model dataframe
    topic_model.get_topic_info().to_json('../data/processed/bertopic_model_small_set_df.json')
    mlflow.log_artifact('../data/processed/bertopic_model_small_set_df.json',
                        artifact_path='bertopic_models')


In [ ]:
# load from MLflow
mlflow_client = mlflow.tracking.MlflowClient(
    tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# load dataframes from artifacts
# mlflow.artifacts.download_artifacts(
#     run_id=mlflow_run_id
# )

# cv_params are parameters for the sklearn CountVectorizer or TFIDFVectorizer
cv_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
    'min_df':10,
}

# bertopic_params are a superset of cv_params
bertopic_params = {
    'top_n_words':20,
    'min_topic_size':10,
    'nr_topics':'auto',
    'verbose':True,
    'low_memory':True,
    'calculate_probabilities':True
}

# update bertopic_params to include cv_params
# bertopic_params.update(cv_params)

# pipeline_params are parameters that will be logged in MLFlow and are a superset of library parameters
pipeline_params = {
    'stanza_model': 'en',
    'sklearn-transformer': 'TfidfVectorizer'
}

# update the pipeline parameters with the library-specific ones so that they show up in MLflow Tracking
pipeline_params.update(cv_params)
pipeline_params.update(bertopic_params)

with mlflow.start_run(experiment_id=get_experiment_id(f"{DAGSHUB_EMAIL}/bertopic_stanza_ingreds_full_set_v1")):    
    # LOG PARAMETERS
    mlflow.log_params(pipeline_params)

    # LOG INPUTS (QUERIES) AND OUTPUTS
    # MLflow example uses a list of strings or a list of str->str dicts
    
    # load raw data and preprocess/clean
    data = dvc.api.read(
           path='../data/recipes-en-201706/epicurious-recipes_m2.json'
           , mode='r')
    raw_df = pd.read_json(data)
    print('\n')
    print('--------------')
    print('Raw Dataframe:', end='\n')
    print(raw_df.head())
    print(raw_df.shape)

    # pre_proc_df is cleaned dataframe
    pre_proc_df = dfpp.preprocess_dataframe(raw_df)
    print('\n')
    print('--------------')
    print('Preprocessed Dataframe:', end='\n')
    print(pre_proc_df.head())
    print(pre_proc_df.shape)


    # pre_proc_df = pd.read_json(
    #     mlflow.artifacts.download_artifacts(
    #         run_id=mlflow_run_id,
    #         artifact_path='artifacts/preprocessed_dataframes/preprocessed_dataframe.json',
    #         # tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow'
    #     )
    # )
    # print('\n')
    # print('-' * 80)
    # print('Preprocessed Dataframe:', end='\n')
    # print(pre_proc_df.head())
    # print(pre_proc_df.shape)

    # create subset for dev purposes
    # to_nlp_df = pre_proc_df[0:50]
    # print('\n')
    # print('-' * 80)
    # print('Subset Dataframe:', end='\n')
    # print(to_nlp_df.head())
    # print(to_nlp_df.shape)

    # LOG MODEL
    # Instantiate BERTopic
    topic_model = BERTopic(
        **bertopic_params,
    )

    def custom_analyzer(step_list, stanza_pipeline, minNgramLength, maxNgramLength):
            lowered = " brk ".join(map(str, [step for step in step_list if step is not None])).lower()

            preproc = stanza_pipeline(lowered)
            
            lemmad = " ".join(map(str,
                                [word.text
                                for sent in preproc.sentences 
                                for word in sent.words if (
                                    word is not None
                                )]
                            )
                        )
            
            # analyze each line of the input string seperately
            for ln in lemmad.split(' brk '):
                
                # tokenize the input string (customize the regex as desired)
                at_least_two_english_characters_whole_words = "(?u)\b[a-zA-Z]{2,}\b"
                terms = re.split(at_least_two_english_characters_whole_words, ln)

                # loop ngram creation for every number between min and max ngram length
                for ngramLength in range(minNgramLength, maxNgramLength+1):

                    # find and return all ngrams
                    # for ngram in zip(*[terms[i:] for i in range(3)]): 
                        # <-- solution without a generator (works the same but has higher memory usage)
                    for ngram in zip(*[islice(seq, i, len(terms)) for i, seq in enumerate(tee(terms, ngramLength))]):   # <-- solution using a generator
                        
                        ngram = ' '.join(map(str, ngram))
                        # yield ngram
                        return str(ngram)

    analyzer_kwargs = {'stanza_pipeline': nlp
                       , 'minNgramLength': 1
                       , 'maxNgramLength': 4}
    
    recipe_ingreds = pre_proc_df["ingredients"].apply(custom_analyzer, **analyzer_kwargs)

    # recipe_steps = "".join(str(to_nlp_df["prepSteps"].apply(StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4))))
    print('\n')
    print('-' * 80)
    print('Recipe ingredients:', end='\n')
    print(recipe_ingreds)

    # train on the recipes' steps
    topics, probs = topic_model.fit_transform(recipe_ingreds)

    # since this uses a custom Stanza analyzer, we have to use a custom mlflow.Pyfunc.PythonModel
    # Instantiate sklearn CountVectorizer
    # steps_vectorizer_model = CountVectorizer(**cv_params)

    # May need to use BERTopic's OnlineCountVectorizer
    steps_vectorizer_model = OnlineCountVectorizer(**cv_params)

    # Do fit transform on data
    # steps_test_tfidf_transform = steps_tfidf_vectorizer_model.fit_transform(tqdm(to_nlp_df["steps"]))
    topic_model.update_topics(
        recipe_ingreds
        , vectorizer_model=steps_vectorizer_model
    )

    # Display topic model results
    print('\n')
    print('-' * 80)
    print('BERTopic Model Dataframe:', end='\n')
    print(topic_model.get_topic_info())

    print('\n')
    print('-' * 80)
    print('BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representation'])

    print('\n')
    print('-' * 80)
    print('BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representative_Docs'])

    # Save and log the topic model dataframe
    topic_model.get_topic_info().to_json('../data/processed/bertopic_model_ingreds_full_set_df.json')
    mlflow.log_artifact('../data/processed/bertopic_model_ingreds_full_set_df.json',
                        artifact_path='bertopic_models')


### Attempt run with lighter weight configuration
#### This attempt will still use Stanza processing on the ingredients 

In [ ]:
# load from MLflow
mlflow_client = mlflow.tracking.MlflowClient(
    tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# cv_params are parameters for the sklearn CountVectorizer or TFIDFVectorizer
sklearn_nlp_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
    'min_df':10,
}

# create sklearn pipeline as in BERTopic lightweight configuration
# pipe = make_pipeline(
#     TfidfVectorizer(**sklearn_nlp_params),
#     TruncatedSVD(100)
# )

# bertopic_params are a superset of cv_params
bertopic_params = {
    # 'embedding_model': TfidfVectorizer(**sklearn_nlp_params),
    'top_n_words':20,
    'min_topic_size':10,
    'nr_topics':50,
    'verbose':True,
    'low_memory':True,
    'calculate_probabilities':True,
    # 'min_cluster_size': 10 # Possibly only works if modifying individual HDBSCAN component of BERTopic
}

# update bertopic_params to include cv_params
# bertopic_params.update(cv_params)

# pipeline_params are parameters that will be logged in MLFlow and are a superset of library parameters
pipeline_params = {
    'stanza_model': 'en',
    'sklearn-transformer': 'TfidfVectorizer'
}

# update the pipeline parameters with the library-specific ones so that they show up in MLflow Tracking
pipeline_params.update(sklearn_nlp_params)
pipeline_params.update(bertopic_params)

with mlflow.start_run(experiment_id=get_experiment_id(f"{DAGSHUB_EMAIL}/bertopic_lightweight_stanza_ingreds_small_set_v1")):    
    # LOG PARAMETERS
    mlflow.log_params(pipeline_params)

    # LOG INPUTS (QUERIES) AND OUTPUTS
    # MLflow example uses a list of strings or a list of str->str dicts
    
    # load raw data and preprocess/clean
    data = dvc.api.read(
           path='../data/recipes-en-201706/epicurious-recipes_m2.json'
           , mode='r')
    raw_df = pd.read_json(data)
    print('\n')
    print('--------------')
    print(f'{datetime.now()}, Raw Dataframe: ', end='\n')
    print(raw_df.head())
    print(raw_df.shape)

    # pre_proc_df is cleaned dataframe
    pre_proc_df = dfpp.preprocess_dataframe(raw_df)
    print('\n')
    print('--------------')
    print(f'{datetime.now()}, Preprocessed Dataframe:', end='\n')
    print(pre_proc_df.head())
    print(pre_proc_df.shape)


    # pre_proc_df = pd.read_json(
    #     mlflow.artifacts.download_artifacts(
    #         run_id=mlflow_run_id,
    #         artifact_path='artifacts/preprocessed_dataframes/preprocessed_dataframe.json',
    #         # tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow'
    #     )
    # )
    # print('\n')
    # print('-' * 80)
    # print('Preprocessed Dataframe:', end='\n')
    # print(pre_proc_df.head())
    # print(pre_proc_df.shape)

    # create subset for dev purposes
    to_nlp_df = pre_proc_df[0:100]
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, Subset Dataframe:', end='\n')
    print(to_nlp_df.head())
    print(to_nlp_df.shape)

    # LOG MODEL
    # Instantiate BERTopic
    topic_model = BERTopic(
        **bertopic_params
    )
    
    analyzer_kwargs = {'stanza_pipeline': nlp
                       , 'minNgramLength': 1
                       , 'maxNgramLength': 4}
    
    recipe_ingreds = to_nlp_df["ingredients"].apply(custom_analyzer, **analyzer_kwargs)

    # Create TF-IDF embeddings
    vectorizer = TfidfVectorizer(**sklearn_nlp_params)
    embeddings = vectorizer.fit_transform(recipe_ingreds)

    # recipe_steps = "".join(str(to_nlp_df["prepSteps"].apply(StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4))))
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, Recipe ingredients:', end='\n')
    print(recipe_ingreds)

    # train on the recipes' ingredientss
    topics, probs = topic_model.fit_transform(recipe_ingreds, embeddings)

    # since this uses a custom Stanza analyzer, we have to use a custom mlflow.Pyfunc.PythonModel
    # Instantiate sklearn CountVectorizer
    sklearn_cv_params = {
        'strip_accents':"unicode",
        'lowercase':True,
        'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
        # 'min_df':10,
    }
    steps_vectorizer_model = CountVectorizer(**sklearn_cv_params)

    # May need to use BERTopic's OnlineCountVectorizer
    # steps_vectorizer_model = OnlineCountVectorizer(**sklearn_nlp_params)

    # Do fit transform on data
    # steps_test_tfidf_transform = steps_tfidf_vectorizer_model.fit_transform(tqdm(to_nlp_df["steps"]))
    topic_model.update_topics(
        recipe_ingreds
        , vectorizer_model=steps_vectorizer_model
    )

    # Display topic model results
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Dataframe:', end='\n')
    print(topic_model.get_topic_info())

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representation'])

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Representative Docs:', end='\n')
    print(topic_model.get_topic_info()['Representative_Docs'])

    # Save and log the topic model dataframe
    topic_model.get_topic_info().to_json('../data/processed/bertopic_model_ingreds_full_set_df.json')
    mlflow.log_artifact('../data/processed/bertopic_model_ingreds_full_set_df.json',
                        artifact_path='bertopic_models')


In [ ]:
topic_model.get_topic_info()['Representation']

In [ ]:
topic_model.get_topic_info()['Representation'][0]

In [ ]:
# load from MLflow
mlflow_client = mlflow.tracking.MlflowClient(
    tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# cv_params are parameters for the sklearn CountVectorizer or TFIDFVectorizer
sklearn_nlp_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
    'min_df':10,
}

# create sklearn pipeline as in BERTopic lightweight configuration
# pipe = make_pipeline(
#     TfidfVectorizer(**sklearn_nlp_params),
#     TruncatedSVD(100)
# )

# bertopic_params are a superset of cv_params
bertopic_params = {
    # 'embedding_model': TfidfVectorizer(**sklearn_nlp_params),
    'top_n_words':20,
    'min_topic_size':10,
    'nr_topics':50,
    'verbose':True,
    'low_memory':True,
    'calculate_probabilities':True,
    # 'min_cluster_size': 10 # Possibly only works if modifying individual HDBSCAN component of BERTopic
}

# update bertopic_params to include cv_params
# bertopic_params.update(cv_params)

# pipeline_params are parameters that will be logged in MLFlow and are a superset of library parameters
pipeline_params = {
    'stanza_model': 'en',
    'sklearn-transformer': 'TfidfVectorizer'
}

# update the pipeline parameters with the library-specific ones so that they show up in MLflow Tracking
pipeline_params.update(sklearn_nlp_params)
pipeline_params.update(bertopic_params)

with mlflow.start_run(experiment_id=get_experiment_id(f"{DAGSHUB_EMAIL}/bertopic_lightweight_stanza_ingreds_small_set_v1")):    
    # LOG PARAMETERS
    mlflow.log_params(pipeline_params)

    # LOG INPUTS (QUERIES) AND OUTPUTS
    # MLflow example uses a list of strings or a list of str->str dicts
    
    # load raw data and preprocess/clean
    data = dvc.api.read(
           path='../data/recipes-en-201706/epicurious-recipes_m2.json'
           , mode='r')
    raw_df = pd.read_json(data)
    print('\n')
    print('--------------')
    print(f'{datetime.now()}, Raw Dataframe: ', end='\n')
    print(raw_df.head())
    print(raw_df.shape)

    # pre_proc_df is cleaned dataframe
    pre_proc_df = dfpp.preprocess_dataframe(raw_df)
    print('\n')
    print('--------------')
    print(f'{datetime.now()}, Preprocessed Dataframe:', end='\n')
    print(pre_proc_df.head())
    print(pre_proc_df.shape)


    # pre_proc_df = pd.read_json(
    #     mlflow.artifacts.download_artifacts(
    #         run_id=mlflow_run_id,
    #         artifact_path='artifacts/preprocessed_dataframes/preprocessed_dataframe.json',
    #         # tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow'
    #     )
    # )
    # print('\n')
    # print('-' * 80)
    # print('Preprocessed Dataframe:', end='\n')
    # print(pre_proc_df.head())
    # print(pre_proc_df.shape)

    # create subset for dev purposes
    to_nlp_df = pre_proc_df[0:100]
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, Subset Dataframe:', end='\n')
    print(to_nlp_df.head())
    print(to_nlp_df.shape)

    # LOG MODEL
    # Instantiate BERTopic
    topic_model = BERTopic(
        **bertopic_params
    )
    
    analyzer_kwargs = {'stanza_pipeline': nlp
                       , 'minNgramLength': 1
                       , 'maxNgramLength': 4}
    
    recipe_ingreds = to_nlp_df["ingredients"].apply(custom_analyzer, **analyzer_kwargs)

    # Create TF-IDF embeddings
    vectorizer = TfidfVectorizer(**sklearn_nlp_params)
    embeddings = vectorizer.fit_transform(recipe_ingreds)

    # recipe_steps = "".join(str(to_nlp_df["prepSteps"].apply(StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4))))
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, Recipe ingredients:', end='\n')
    print(recipe_ingreds)

    # train on the recipes' ingredientss
    topics, probs = topic_model.fit_transform(recipe_ingreds, embeddings)

    # since this uses a custom Stanza analyzer, we have to use a custom mlflow.Pyfunc.PythonModel
    # Instantiate sklearn CountVectorizer
    sklearn_cv_params = {
        'strip_accents':"unicode",
        'lowercase':True,
        # 'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
        # 'min_df':10,
    }
    steps_vectorizer_model = CountVectorizer(**sklearn_cv_params)

    # May need to use BERTopic's OnlineCountVectorizer
    # steps_vectorizer_model = OnlineCountVectorizer(**sklearn_nlp_params)

    # Do fit transform on data
    # steps_test_tfidf_transform = steps_tfidf_vectorizer_model.fit_transform(tqdm(to_nlp_df["steps"]))
    topic_model.update_topics(
        recipe_ingreds
        , vectorizer_model=steps_vectorizer_model
    )

    # Display topic model results
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Dataframe:', end='\n')
    print(topic_model.get_topic_info())

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representation'])

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Representative Docs:', end='\n')
    print(topic_model.get_topic_info()['Representative_Docs'])

    # Save and log the topic model dataframe
    topic_model.get_topic_info().to_json('../data/processed/bertopic_model_ingreds_full_set_df.json')
    mlflow.log_artifact('../data/processed/bertopic_model_ingreds_full_set_df.json',
                        artifact_path='bertopic_models')


In [ ]:
# load from MLflow
mlflow_client = mlflow.tracking.MlflowClient(
    tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# cv_params are parameters for the sklearn CountVectorizer or TFIDFVectorizer
sklearn_nlp_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
    'min_df':10,
}

# create sklearn pipeline as in BERTopic lightweight configuration
# pipe = make_pipeline(
#     TfidfVectorizer(**sklearn_nlp_params),
#     TruncatedSVD(100)
# )

# bertopic_params are a superset of cv_params
bertopic_params = {
    # 'embedding_model': TfidfVectorizer(**sklearn_nlp_params),
    'top_n_words':20,
    'min_topic_size':10,
    'nr_topics':50,
    'verbose':True,
    'low_memory':True,
    'calculate_probabilities':True,
    # 'min_cluster_size': 10 # Possibly only works if modifying individual HDBSCAN component of BERTopic
}

# update bertopic_params to include cv_params
# bertopic_params.update(cv_params)

# pipeline_params are parameters that will be logged in MLFlow and are a superset of library parameters
pipeline_params = {
    'stanza_model': 'en',
    'sklearn-transformer': 'TfidfVectorizer'
}

# update the pipeline parameters with the library-specific ones so that they show up in MLflow Tracking
pipeline_params.update(sklearn_nlp_params)
pipeline_params.update(bertopic_params)

with mlflow.start_run(experiment_id=get_experiment_id(f"{DAGSHUB_EMAIL}/bertopic_lightweight_stanza_ingreds_small_set_v1.01")):    
    # LOG PARAMETERS
    mlflow.log_params(pipeline_params)

    # LOG INPUTS (QUERIES) AND OUTPUTS
    # MLflow example uses a list of strings or a list of str->str dicts
    
    # load raw data and preprocess/clean
    data = dvc.api.read(
           path='../data/recipes-en-201706/epicurious-recipes_m2.json'
           , mode='r')
    raw_df = pd.read_json(data)
    print('\n')
    print('--------------')
    print(f'{datetime.now()}, Raw Dataframe: ', end='\n')
    print(raw_df.head())
    print(raw_df.shape)

    # pre_proc_df is cleaned dataframe
    pre_proc_df = dfpp.preprocess_dataframe(raw_df)
    print('\n')
    print('--------------')
    print(f'{datetime.now()}, Preprocessed Dataframe:', end='\n')
    print(pre_proc_df.head())
    print(pre_proc_df.shape)


    # pre_proc_df = pd.read_json(
    #     mlflow.artifacts.download_artifacts(
    #         run_id=mlflow_run_id,
    #         artifact_path='artifacts/preprocessed_dataframes/preprocessed_dataframe.json',
    #         # tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow'
    #     )
    # )
    # print('\n')
    # print('-' * 80)
    # print('Preprocessed Dataframe:', end='\n')
    # print(pre_proc_df.head())
    # print(pre_proc_df.shape)

    # create subset for dev purposes
    to_nlp_df = pre_proc_df[0:100]
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, Subset Dataframe:', end='\n')
    print(to_nlp_df.head())
    print(to_nlp_df.shape)

    # LOG MODEL
    # Instantiate BERTopic
    topic_model = BERTopic(
        **bertopic_params
    )
    
    analyzer_kwargs = {'stanza_pipeline': nlp
                       , 'minNgramLength': 1
                       , 'maxNgramLength': 4
                       , 'lemmatize': True}
    
    # recipe_steps = "".join(str(to_nlp_df["prepSteps"].apply(StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4))))
    recipe_ingreds = to_nlp_df["ingredients"].apply(custom_analyzer, **analyzer_kwargs)

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, Recipe ingredients:', end='\n')
    print([ingred for ingred in recipe_ingreds])

    # Create TF-IDF embeddings
    vectorizer = TfidfVectorizer(**sklearn_nlp_params)
    embeddings = vectorizer.fit_transform(tqdm(recipe_ingreds))

    # train on the recipes' ingredientss
    topics, probs = topic_model.fit_transform(recipe_ingreds, embeddings)

    # since this uses a custom Stanza analyzer, we have to use a custom mlflow.Pyfunc.PythonModel
    # Instantiate sklearn CountVectorizer
    sklearn_cv_params = {
        # 'strip_accents':"unicode",
        # 'lowercase':True,
        # 'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
        # 'min_df':10,
        'token_pattern': r"(?u)\b[a-zA-Z]{2,}\b"
    }
    ingreds_vectorizer_model = CountVectorizer(**sklearn_cv_params)

    # May need to use BERTopic's OnlineCountVectorizer
    # steps_vectorizer_model = OnlineCountVectorizer(**sklearn_nlp_params)

    # Do fit transform on data
    # steps_test_tfidf_transform = steps_tfidf_vectorizer_model.fit_transform(tqdm(to_nlp_df["steps"]))
    topic_model.update_topics(
        recipe_ingreds
        , vectorizer_model=ingreds_vectorizer_model
    )

    # Display topic model results
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Dataframe:', end='\n')
    print(topic_model.get_topic_info())

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representation'])

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Representative Docs:', end='\n')
    print(topic_model.get_topic_info()['Representative_Docs'])

    # Save and log the topic model dataframe
    topic_model.get_topic_info().to_json('../data/processed/bertopic_model_ingreds_small_set_df.json')
    mlflow.log_artifact('../data/processed/bertopic_model_ingreds_small_set_df.json',
                        artifact_path='bertopic_models')




--------------
2023-12-18 18:13:41.917123, Raw Dataframe: 
                         id  \
0  54a2b6b019925f464b373351   
1  54a408a019925f464b3733bc   
2  54a408a26529d92b2c003631   
3  54a408a66529d92b2c003638   
4  54a408a719925f464b3733cc   

                                                 dek  \
0  How does fried chicken achieve No. 1 status? B...   
1                                Spinaci all'Ebraica   
2  This majestic, moist, and richly spiced honey ...   
3  The idea for this sandwich came to me when my ...   
4  In 1930, Simon Agranat, the chief justice of t...   

                                     hed                   pubDate  \
0            Pickle-Brined Fried Chicken  2014-08-19T04:00:00.000Z   
1                   Spinach Jewish Style  2008-09-09T04:00:00.000Z   
2                  New Year’s Honey Cake  2008-09-10T04:00:00.000Z   
3  The B.L.A.Bagel with Lox and Avocado  2008-09-08T04:00:00.000Z   
4        Shakshuka a la Doktor Shakshuka  2008-09-09T04:00:00.000

100%|██████████| 100/100 [03:51<00:00,  2.31s/it]
2023-12-18 18:23:57,887 - BERTopic - Reduced dimensionality
2023-12-18 18:23:57,895 - BERTopic - Clustered reduced embeddings
2023-12-18 18:23:57,908 - BERTopic - Reduced number of topics from 5 to 5




--------------------------------------------------------------------------------
2023-12-18 18:23:57.932116, BERTopic Model Dataframe:
   Topic  Count                            Name  \
0     -1      6     -1_sugar_chop_spinach_leave   
1      0     12    0_flour_purpose_cup_mushroom   
2      1     55     1_tablespoon_large_cup_seed   
3      2     14  2_oil_nonstick_spray_vegetable   
4      3     13       3_fish_fillet_such_salmon   

                                      Representation  \
0  [sugar, chop, spinach, leave, carrot, bulk, ba...   
1  [flour, purpose, cup, mushroom, porcini, tomat...   
2  [tablespoon, large, cup, seed, pound, butter, ...   
3  [oil, nonstick, spray, vegetable, olive, milk,...   
4  [fish, fillet, such, salmon, olive, tablespoon...   

                                 Representative_Docs  
0  [cup sugar, bacon slice chop, pound small leav...  
1  [cup purpose flour, cup purpose flour, cup pur...  
2  [cup inch cube peel seed butternut squash poun...  

In [ ]:
to_nlp_df['ingredients'][0]

['1 tablespoons yellow mustard seeds',
 '1 tablespoons brown mustard seeds',
 '1 1/2 teaspoons coriander seeds',
 '1 cup apple cider vinegar',
 '2/3 cup kosher salt',
 '1/3 cup sugar',
 '1/4 cup chopped fresh dill',
 '8 skinless, boneless chicken thighs (about 3 pounds), halved, quartered if large',
 'Vegetable oil (for frying; about 10 cups)',
 '2 cups buttermilk',
 '2 cups all-purpose flour',
 'Kosher salt',
 'Honey, flaky sea salt (such as Maldon), toasted benne or sesame seeds, hot sauce (for serving)',
 'A deep-fry thermometer']

In [ ]:
test_recipe_ingreds = to_nlp_df["ingredients"].apply(custom_analyzer, **analyzer_kwargs)

tablespoon yellow mustard seed
pound small - leave bulk spinach
cup - purpose flour


KeyboardInterrupt: 

i think i should start leaving out units/including stopwords again since i'm not using Stanza's deep learning

In [ ]:
# load from MLflow
mlflow_client = mlflow.tracking.MlflowClient(
    tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# cv_params are parameters for the sklearn CountVectorizer or TFIDFVectorizer
sklearn_nlp_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
    'min_df':10,
}

# bertopic_params are a superset of cv_params
bertopic_params = {
    'top_n_words':20,
    'min_topic_size':10,
    'nr_topics':50,
    'verbose':True,
    'low_memory':True,
    'calculate_probabilities':True,
    # 'min_cluster_size': 10 # Possibly only works if modifying individual HDBSCAN component of BERTopic
}

# update bertopic_params to include cv_params
# bertopic_params.update(cv_params)

# pipeline_params are parameters that will be logged in MLFlow and are a superset of library parameters
pipeline_params = {
    'stanza_model': 'en',
    'sklearn-transformer': 'TfidfVectorizer'
}

# update the pipeline parameters with the library-specific ones so that they show up in MLflow Tracking
pipeline_params.update(sklearn_nlp_params)
pipeline_params.update(bertopic_params)

with mlflow.start_run(experiment_id=get_experiment_id(f"{DAGSHUB_EMAIL}/bertopic_lightweight_stanza_ingreds_full_set_v1.00")):    
    # LOG PARAMETERS
    mlflow.log_params(pipeline_params)

    # LOG INPUTS (QUERIES) AND OUTPUTS
    # MLflow example uses a list of strings or a list of str->str dicts
    
    # load raw data and preprocess/clean
    data = dvc.api.read(
           path='../data/recipes-en-201706/epicurious-recipes_m2.json'
           , mode='r')
    raw_df = pd.read_json(data)
    print('\n')
    print('--------------')
    print(f'{datetime.now()}, Raw Dataframe: ', end='\n')
    print(raw_df.head())
    print(raw_df.shape)

    # pre_proc_df is cleaned dataframe
    pre_proc_df = dfpp.preprocess_dataframe(raw_df)
    print('\n')
    print('--------------')
    print(f'{datetime.now()}, Preprocessed Dataframe:', end='\n')
    print(pre_proc_df.head())
    print(pre_proc_df.shape)


    # pre_proc_df = pd.read_json(
    #     mlflow.artifacts.download_artifacts(
    #         run_id=mlflow_run_id,
    #         artifact_path='artifacts/preprocessed_dataframes/preprocessed_dataframe.json',
    #         # tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow'
    #     )
    # )
    # print('\n')
    # print('-' * 80)
    # print('Preprocessed Dataframe:', end='\n')
    # print(pre_proc_df.head())
    # print(pre_proc_df.shape)

    # create subset for dev purposes
    # to_nlp_df = pre_proc_df[0:100]
    # print('\n')
    # print('-' * 80)
    # print(f'{datetime.now()}, Subset Dataframe:', end='\n')
    # print(to_nlp_df.head())
    # print(to_nlp_df.shape)

    # LOG MODEL
    # Instantiate BERTopic
    topic_model = BERTopic(
        **bertopic_params
    )
    
    analyzer_kwargs = {'stanza_pipeline': nlp
                       , 'minNgramLength': 1
                       , 'maxNgramLength': 4
                       , 'lemmatize': True}
    
    recipe_ingreds = pre_proc_df["ingredients"].apply(custom_analyzer, **analyzer_kwargs)
    
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, Recipe ingredients:', end='\n')
    print(recipe_ingreds)

    # Create TF-IDF embeddings
    vectorizer = TfidfVectorizer(**sklearn_nlp_params)
    embeddings = vectorizer.fit_transform(tqdm(recipe_ingreds))

    # recipe_steps = "".join(str(to_nlp_df["prepSteps"].apply(StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4))))
    # print('\n')
    # print('-' * 80)
    # print(f'{datetime.now()}, Recipe ingredients:', end='\n')
    # print(recipe_ingreds)

    # train on the recipes' ingredientss
    topics, probs = topic_model.fit_transform(recipe_ingreds, embeddings)

    # since this uses a custom Stanza analyzer, we have to use a custom mlflow.Pyfunc.PythonModel
    # Instantiate sklearn CountVectorizer
    sklearn_cv_params = {
        # 'strip_accents':"unicode",
        # 'lowercase':True,
        'token_pattern': r"(?u)\b[a-zA-Z]{2,}\b"
    }
    ingreds_vectorizer_model = CountVectorizer(**sklearn_cv_params)

    # Do fit transform on data
    # steps_test_tfidf_transform = steps_tfidf_vectorizer_model.fit_transform(tqdm(to_nlp_df["steps"]))
    topic_model.update_topics(
        recipe_ingreds
        , vectorizer_model=ingreds_vectorizer_model
    )

    # Display topic model results
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Dataframe:', end='\n')
    print(topic_model.get_topic_info())

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representation'])

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Representative Docs:', end='\n')
    print(topic_model.get_topic_info()['Representative_Docs'])

    # Save and log the topic model dataframe
    topic_model.get_topic_info().to_json('../data/processed/bertopic_model_ingreds_full_set_df.json')
    mlflow.log_artifact('../data/processed/bertopic_model_ingreds_full_set_df.json',
                        artifact_path='bertopic_models')




--------------
2023-12-19 20:08:42.203994, Raw Dataframe: 
                         id  \
0  54a2b6b019925f464b373351   
1  54a408a019925f464b3733bc   
2  54a408a26529d92b2c003631   
3  54a408a66529d92b2c003638   
4  54a408a719925f464b3733cc   

                                                 dek  \
0  How does fried chicken achieve No. 1 status? B...   
1                                Spinaci all'Ebraica   
2  This majestic, moist, and richly spiced honey ...   
3  The idea for this sandwich came to me when my ...   
4  In 1930, Simon Agranat, the chief justice of t...   

                                     hed                   pubDate  \
0            Pickle-Brined Fried Chicken  2014-08-19T04:00:00.000Z   
1                   Spinach Jewish Style  2008-09-09T04:00:00.000Z   
2                  New Year’s Honey Cake  2008-09-10T04:00:00.000Z   
3  The B.L.A.Bagel with Lox and Avocado  2008-09-08T04:00:00.000Z   
4        Shakshuka a la Doktor Shakshuka  2008-09-09T04:00:00.000

100%|██████████| 34656/34656 [10:31:18<00:00,  1.09s/it]  
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.10/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
2023-12-27 02:35:54,492 - BERTopic - Reduced dimensionality
2023-12-27 02:45:04,427 - BERTopic - Clustered reduced embeddings


ValueError: Found array with dim 3. check_pairwise_arrays expected <= 2.

In [ ]:
# try splitting among CPU and GPU. Try Stanza on CPU due to its memory usage
nlp2 = stanza.Pipeline('en', use_gpu=False)

